# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
#upgrade the sdk version by running below command in jupyter notebook terminal window 
#and restart the kernal before running this command.

#pip install --upgrade azureml-core
#pip install --upgrade azureml-automl-core
#pip install --upgrade --upgrade-strategy eager azureml-sdk[automl,widgets,notebooks]

SyntaxError: invalid syntax (<ipython-input-6-36cc90423c67>, line 4)

In [45]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
import azureml.train.automl
from azureml.pipeline.steps import AutoMLStep



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')



quick-starts-ws-135768
aml-quickstarts-135768
southcentralus
a24a24d5-8d87-4c8a-99b6-91ed2d2df51f


In [6]:
# choose a name for experiment
experiment_name = 'capstone-Automl'
#project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment
run = experiment.start_logging()

# Creat Compute Cluster

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your GPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().
compute_target.get_status()

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory


In [9]:
datastore= ws.get_default_datastore()

In [10]:
datastore.upload(src_dir= './', target_path= './')

Uploading an estimated of 8 files
Uploading ./data.json
Uploaded ./data.json, 1 files out of an estimated total of 8
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 2 files out of an estimated total of 8
Uploading ./heart_failure_clinical_records_dataset.csv
Uploaded ./heart_failure_clinical_records_dataset.csv, 3 files out of an estimated total of 8
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 4 files out of an estimated total of 8
Uploading ./model-register-and-deploy.yml
Uploaded ./model-register-and-deploy.yml, 5 files out of an estimated total of 8
Uploading ./score.py
Uploaded ./score.py, 6 files out of an estimated total of 8
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 7 files out of an estimated total of 8
Uploading ./.ipynb_checkpoints/hyperparameter_tuning-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/hyperparameter_tuning-checkpoint.ipynb, 8 files out of an estimated total

$AZUREML_DATAREFERENCE_workspaceblobstore

In [11]:
tds= Dataset.Tabular.from_delimited_files(datastore.path('./heart_failure_clinical_records_dataset.csv'))

In [12]:
tds.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [42]:
df= tds.to_pandas_dataframe()

In [13]:
dataset_x = tds.to_pandas_dataframe().drop('DEATH_EVENT', axis= 1)
dataset_x.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=tds,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
# Retrieve and save your best automl model.
remote_run.wait_for_completion(show_output=True)




Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

{'runId': 'AutoML_e6cb8a55-f8ac-403a-8689-2139762b0d4b',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-23T10:22:08.389283Z',
 'endTimeUtc': '2021-01-23T10:50:38.445592Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-Automl","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-135768","workspace_name":"quick-starts-ws-135768","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuri

In [17]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
#TODO: Save the best model
best_run, fitted_model = remote_run.get_output()

best_run_metrics = best_run.get_metrics()

print(fitted_mode, best_run.id, best_run_metrics['accuracy'])


AttributeError: Can't get attribute 'XGBoostLabelEncoder' on <module 'xgboost.compat' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/compat.py'>

In [20]:
#print(best_run)
print(fitted_model)

NameError: name 'fitted_model' is not defined

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
description = 'HearFailure_AutoML_Model'
model = remote_run.register_model(description = description)


In [24]:
print(model.name, model.id, model.version, model.description, sep ='\t')

AutoMLe6cb8a55f53	AutoMLe6cb8a55f53:1	1	HearFailure_AutoML_Model


In [27]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-135768', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-135768'), name=AutoMLe6cb8a55f53, id=AutoMLe6cb8a55f53:1, version=1, tags={}, properties={})


In [28]:
import joblib

joblib.dump(model, 'outputs/HearFailure_AutoML_Model.pkl')

TypeError: can't pickle _thread.RLock objects

In [29]:
from azureml.core.model import Model

model = Model.register(workspace=ws,
                       model_name='HearFailure_AutoML_Model',                # Name of the registered model in your workspace.
                       model_path='outputs/HearFailure_AutoML_Model.pkl',  # Local file to upload and register as a model.
                       description='Heart Failure death prediction',
                       tags={'area': 'heart failure', 'type': 'Classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model HearFailure_AutoML_Model
Name: HearFailure_AutoML_Model
Version: 1


In [31]:
%%writefile env.yml
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment.get(ws, "AzureML-AutoML-GPU")
env.inferencing_stack_version='latest'

#for pip_package in ["scikit-learn"]:
#    env.python.conda_dependencies.add_pip_package(pip_package)



Writing env.yml


In [54]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

In [56]:
from azureml.core.webservice import AciWebservice 
from azureml.core import Model

dep_cfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights= True, auth_enabled= True) 

webservice = Model.deploy(ws, "capstone-automl1", [model], inference_config, dep_cfg)

webservice.wait_for_deployment(show_output = True) 
print(webservice.state) 


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: d329ed3b-b6e3-4493-9a8e-ff37fb3ed31b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: capstone-automl1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_6b386f93e6a650a8cca5338d7fb60982 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: d329ed3b-b6e3-4493-9a8e-ff37fb3ed31b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: capstone-automl1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_6b386f93e6a650a8cca5338d7fb60982 locally. Please refer to https://aka.ms/debugimage#service-launc

In [41]:
print(webservice.get_logs())
print(webservice.scoring_uri) 
print(webservice.swagger_uri)

None
None
None


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json


input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = webservice.run(input_payload)
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
webservice.delete()

NameError: name 'webservice' is not defined